In [1]:
import pandas as pd
import zipfile
import os
import time
from tqdm import tqdm
import shutil
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [3]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add
path="./"
files_add=find_the_way(path,'.pcap')
files_add

['./pcaps\\dos-synflooding-1-dec.pcap',
 './pcaps\\dos-synflooding-2-dec.pcap',
 './pcaps\\dos-synflooding-3-dec.pcap',
 './pcaps\\dos-synflooding-4-dec.pcap',
 './pcaps\\dos-synflooding-5-dec.pcap',
 './pcaps\\dos-synflooding-6-dec.pcap',
 './pcaps\\mirai-ackflooding-1-dec.pcap',
 './pcaps\\mirai-ackflooding-2-dec.pcap',
 './pcaps\\mirai-ackflooding-3-dec.pcap',
 './pcaps\\mirai-ackflooding-4-dec.pcap',
 './pcaps\\mirai-hostbruteforce-1-dec.pcap',
 './pcaps\\mirai-hostbruteforce-2-dec.pcap',
 './pcaps\\mirai-hostbruteforce-3-dec.pcap',
 './pcaps\\mirai-hostbruteforce-4-dec.pcap',
 './pcaps\\mirai-hostbruteforce-5-dec.pcap',
 './pcaps\\mirai-httpflooding-1-dec.pcap',
 './pcaps\\mirai-httpflooding-2-dec.pcap',
 './pcaps\\mirai-httpflooding-3-dec.pcap',
 './pcaps\\mirai-httpflooding-4-dec.pcap',
 './pcaps\\mirai-udpflooding-1-dec.pcap',
 './pcaps\\mirai-udpflooding-2-dec.pcap',
 './pcaps\\mirai-udpflooding-3-dec.pcap',
 './pcaps\\mirai-udpflooding-4-dec.pcap',
 './pcaps\\Mirai.pcap',
 '.

In [4]:
len(files_add)

43

# PCAP2CSV

In [5]:
from scapy.all import*

In [6]:
import math
import pandas as pd
import os
import numpy as np

In [7]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [8]:
def port_class(port):
    port_list=[0,53,67,68,80,123,443,1900,5353,49153]
    if port in port_list:
        return port_list.index(port)+1
    elif 0 <= port <= 1023:
        return 11
    elif  1024 <= port <= 49151 :
        return 12
    elif 49152 <=port <= 65535 :
        return 13
    else:
        return 0
    
    


In [9]:
header="ts,pck_size,IP_id,ICMP_chksum,ICMP_id,TCP_seq,TCP_ack,TCP_chksum,UDP_chksum,DNS_id,BOOTP_xid,sport,dport,sport_class,dport_class,Label\n"

# Individual paket pased faetures

In [10]:
def pcap_parser(oldname):
    piece_size=50000
    new_name=oldname.replace(".pcap","_smaller.pcap")
    command='C:/\"Program Files\"/Wireshark/editcap.exe -c '+str(piece_size)+" \""+oldname+"\" "+new_name   
    os.system(command)
    parsed=find_the_way("./",new_name[6:-5])
    return parsed,new_name[6:-5]
    

In [11]:
def merged_csv(name,keyword):
    for merger in ["_FE.csv","_WS.csv"]:
        merged_name=f"{name[:-4]}{merger}"
        csv_files=find_the_way("./",keyword,merger)
        df=pd.read_csv(csv_files[0])
        col_names=list(df.columns)

        empty = pd.DataFrame(columns=col_names)
        empty.to_csv(merged_name, mode="w", index=False)#,header=False)


        for j in csv_files:

            df=pd.read_csv(j)
            #print("name and shape of dataframe :",i,df.shape)
            df.to_csv(merged_name, mode="a", index=False,header=False)


            os.remove(j)
            try:
                os.remove(j.replace("_FE.csv",".pcap"))
            except:pass

            #print("\n\n\nname and shape of dataframe :",name,df.shape)

In [12]:
outputfolder="FE"
folder(outputfolder)
for sayac,i in enumerate (files_add):
    print(f"{sayac+1}/{len(files_add)}-{i}")
    try:
        csv=i.replace(".pcap",".csv")
        df=pd.read_csv(csv)
        df=df["Label"].values
        label=True
    except:
        print("Predetermined tag not found")
        label=False
    boyut=os.path.getsize(i)
    label_count=0  
    for isim in [i]:
        filename=isim.replace(".pcap","_FE.csv")
        ths = open(filename, "w")
        ths.write(header)  
        pkt = rdpcap(isim)
        for j in tqdm(pkt):
            ts=float(j.time)
            try:pck_size=j.len
            except:pck_size=0
            if j.haslayer(IP):
                IP_id=j[IP].id
                IP_chksum=j[IP].chksum
            else:
                IP_id=0
                IP_chksum=0
            if j.haslayer(ICMP):
                ICMP_chksum=j[ICMP].chksum
                ICMP_id=j[ICMP].id
            else:
                ICMP_chksum=0
                ICMP_id=0
            if j.haslayer(TCP):
                TCP_sport=j[TCP].sport
                TCP_dport=j[TCP].dport
                TCP_seq=j[TCP].seq
                TCP_ack=j[TCP].ack
                TCP_chksum=j[TCP].chksum
            else:
                TCP_sport=0
                TCP_dport=0
                TCP_seq=0
                TCP_ack=0
                TCP_chksum=0
            if j.haslayer(UDP):
                UDP_sport=j[UDP].sport
                UDP_dport=j[UDP].dport
                UDP_chksum=j[UDP].chksum
            else:
                UDP_sport=0
                UDP_dport=0
                UDP_chksum=0
            if j.haslayer(BOOTP):
                BOOTP_xid=j[BOOTP].xid
            else:
                BOOTP_xid=0
            if j.haslayer(DNS):

                DNS_id=j[DNS].id
            else:
                DNS_id=0
            sport_class=port_class(TCP_sport+UDP_sport)
            dport_class=port_class(TCP_dport+UDP_dport)
       
            sport=TCP_sport+UDP_sport
            dport=TCP_dport+UDP_dport#port_class(TCP_dport+UDP_dport)
            try:Mac=j.src  
            except:Mac= j.addr1     
            line=[ts,pck_size,IP_id,ICMP_chksum,ICMP_id,TCP_seq,TCP_ack,TCP_chksum,UDP_chksum,DNS_id,BOOTP_xid,sport,dport,sport_class,dport_class]
            line=str(line).replace("[","")
            line=str(line).replace("]","")
            #line=str(line).replace("\',","-")
            line=str(line).replace(", ",",")
            line=str(line).replace("\'","")
            line=str(line).replace("None","0")
            if label:
                Label=df[label_count]
            else:
                Label="No_Label"
            ths.write(str(line)+f",{Label}\n")  
            label_count+=1
        ths.close()          

        
        

1/43-./pcaps\dos-synflooding-1-dec.pcap


100%|██████████████████████████████████████████████████████████████████████████| 40788/40788 [00:05<00:00, 7527.93it/s]


2/43-./pcaps\dos-synflooding-2-dec.pcap


100%|██████████████████████████████████████████████████████████████████████████| 39742/39742 [00:05<00:00, 7353.91it/s]


3/43-./pcaps\dos-synflooding-3-dec.pcap


...

36/43-./pcaps\scan-hostport-6-dec.pcap


100%|████████████████████████████████████████████████████████████████████████████| 6828/6828 [00:00<00:00, 7621.20it/s]


37/43-./pcaps\scan-portos-1-dec.pcap


 99%|█████████████████████████████████████████████████████████████████████████▌| 72760/73235 [00:09<00:00, 7337.38it/s]


IndexError: index 72760 is out of bounds for axis 0 with size 72760